# Here be dragons.
### Before you do anything
* Read everything below.
* Download a complete, fresh copy of the repo you want to shrink.
* Copy that repo again on your computer.
* Run this carefully on a copy of the newly copied repo. Copy?

### What this does
This attempts to take the non-main/non-master branches of a Git repo that's been bloated through excessive snapshots of Github Actions. It aims to substantially shrink the size of repos.

It builds a branch of each of those original branches; commits those files; deletes the original branch; and uploads the replacement in its stead.

This in effect flattens the repo and kills off all the history, including older snapshots of working files that have been getting diff'd.

In [ ]:
import sys
!{sys.executable} -m pip install GitPython

In [ ]:
from git import *

In [ ]:
untouchables = ["origin/HEAD", "origin/main", "origin/master"]    # What branches should never be trimmed?

In [ ]:
repo = Repo('.')
assert not repo.bare

In [ ]:
git = repo.git

In [ ]:
remote_refs = repo.remote().refs

In [ ]:
branchestotrim = []

In [ ]:
for remote_ref in remote_refs:
    if remote_ref.name not in untouchables:
        branchestotrim.append(remote_ref.name)

In [ ]:
branchestotrim

In [ ]:
for branchtotrim in branchestotrim:
    git.checkout(branchtotrim)
    localname = "local-" + branchtotrim.split("/")[-1]
    branchhost = "/".join(branchtotrim.split("/")[:-1])    
    branchtarget = branchtotrim.split("/")[-1]
    print(f"Working on {branchtotrim} at {localname}")
    git.checkout("--orphan", localname)
    git.add("-A")     # Add all files
    changedFiles = repo.index.diff("HEAD")
    if not changedFiles:      # Delete temporary branch and do nothing else
        git.checkout(branchtotrim)
        git.branch("-D", localname)
    else:              # Files changed; replace old branch with temporary branch
        git.commit("-m", f"Cleaning up {branchtotrim}")
        git.branch("-D", branchtotrim)    # Delete old branch
        git.branch("-m", branchtotrim)    # Move temporary branch into its place
        git.push("-u", "-f", branchhost, branchtarget)    # And upload